In [1]:
import numpy as onp
import jax.numpy as jnp
from jax import grad, jit, vmap, value_and_grad
import jax

# Generate key which is used to generate random numbers
init_rng_key = jax.random.PRNGKey(1)




/usr/local/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [198]:
n_samples, n_dims = 1000, 20

x = jnp.array(onp.random.randn(n_samples, n_dims))
w_true = jnp.array(onp.random.randn(n_dims))
y_true = jnp.dot(x, w_true)
print(x.shape)
print(w_true.shape)
print(y_true.shape)
w_init = jnp.array(onp.random.randn(n_dims))
print(w.shape)
true_params = {"w": w_true}
params = {"w": w_init}

print(params)

(1000, 20)
(20,)
(1000,)
(20,)
{'w': DeviceArray([-0.7944283 , -0.6737215 ,  1.0270158 ,  0.93113774,
              0.40253142, -1.6318681 , -0.58861333, -0.7433417 ,
              0.5095647 , -0.9508537 ,  0.10090663,  2.105676  ,
             -1.6571679 , -0.38133493, -0.93313324,  0.17004208,
              0.8761585 , -0.31096017, -0.1065815 , -0.08023604],            dtype=float32)}


In [207]:
def forward_one(xi, w):
    return jnp.dot(xi, w)

def forward_vmap(x, w):
    return jax.vmap(lambda xi: forward_one(xi, w),)(x)

def forward(x, w):
    return jnp.dot(x, w) 

y_pred_init = forward(x, w)
print(forward_one(x[0, :], w).shape)
print(y_pred_init.shape)


def mse(params, x, y_true):
    y_pred = jnp.dot(x, params["w"]) 
    diff = y_true - y_pred 
    return jnp.mean(diff**2)

print(mse(params, x, y_true))

dloss = jax.grad(mse)

print(w_init)
print(w_true)
print(dloss(true_params, x, y_true))
print(dloss(params, x, y_true))

def full_pass(params, x, y_true, lr):
    gradients = dloss(params, x, y_true)
    dw = gradients["w"]
    print("dw", dw)
    new_w = w - lr * dw
    return {"w": new_w}

def opt(params, x, y_true, lr=0.05, iters=5000):
    for i in range(iters):
        old_w = params["w"]
        params = full_pass(params, x, y_true, lr=lr)
        y_pred = forward(x, params["w"])
        print( i, ((y_pred-y_true) ** 2).mean())
        print(params["w"])
    return params


()
(1000,)
45.192886
[-0.7944283  -0.6737215   1.0270158   0.93113774  0.40253142 -1.6318681
 -0.58861333 -0.7433417   0.5095647  -0.9508537   0.10090663  2.105676
 -1.6571679  -0.38133493 -0.93313324  0.17004208  0.8761585  -0.31096017
 -0.1065815  -0.08023604]
[ 1.2244141  -2.0427547   0.41138938 -0.00851617 -0.17035753  0.7279684
 -0.17386378  0.8345659  -0.42434725  0.12314823 -0.0817145   0.16684777
  1.082163   -0.3168161   0.7444031  -0.28579453 -0.7162781   0.9167828
  0.3207458   2.6583536 ]
{'w': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0.], dtype=float32)}
{'w': DeviceArray([-4.446989  ,  2.9414291 ,  1.7367637 ,  2.1285372 ,
              1.6311722 , -4.5992455 , -0.63259214, -2.9020622 ,
              0.7663313 , -1.9877609 ,  0.7132008 ,  3.6403208 ,
             -6.418166  , -0.02409841, -3.2791257 ,  1.3718442 ,
              3.182522  , -2.7029574 , -1.0877582 , -5.5075746 ],            dtype=float32)}


In [208]:
print(true_params)
new_params = opt(params, x, y_true)


{'w': DeviceArray([ 1.2244141 , -2.0427547 ,  0.41138938, -0.00851617,
             -0.17035753,  0.7279684 , -0.17386378,  0.8345659 ,
             -0.42434725,  0.12314823, -0.0817145 ,  0.16684777,
              1.082163  , -0.3168161 ,  0.7444031 , -0.28579453,
             -0.7162781 ,  0.9167828 ,  0.3207458 ,  2.6583536 ],            dtype=float32)}
{'w': DeviceArray([-0.7944283 , -0.6737215 ,  1.0270158 ,  0.93113774,
              0.40253142, -1.6318681 , -0.58861333, -0.7433417 ,
              0.5095647 , -0.9508537 ,  0.10090663,  2.105676  ,
             -1.6571679 , -0.38133493, -0.93313324,  0.17004208,
              0.8761585 , -0.31096017, -0.1065815 , -0.08023604],            dtype=float32)}
dw [-4.446989    2.9414291   1.7367637   2.1285372   1.6311722  -4.5992455
 -0.63259214 -2.9020622   0.7663313  -1.9877609   0.7132008   3.6403208
 -6.418166   -0.02409841 -3.2791257   1.3718442   3.182522   -2.7029574
 -1.0877582  -5.5075746 ]
0 14.227691
{'w': DeviceArray([ 0.223

dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
23 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
24 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409, 

43 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
44 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547

63 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
64 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547

dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
80 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
81 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409, 

...

dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4936 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4937 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.002824

4953 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4954 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2

4968 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4969 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2

{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4985 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.94

dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4998 15.236733
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.06674194,  0.0928847 ,  0.04256445,  0.23481317],            dtype=float32)}
dw [-2.547898    3.9434202  -0.44025648 -0.1527591   0.44185093 -1.3841156
  0.51564044 -1.652571    0.3878682  -0.20368037  0.2212831  -0.47115096
 -2.300533    0.6028258  -1.0990205   0.76361984  1.249017   -1.7387776
 -0.86286855 -4.81465   ]
4999 15.236733


In [209]:
print(params)
print(true_params)
print(new_params)

{'w': DeviceArray([-0.7944283 , -0.6737215 ,  1.0270158 ,  0.93113774,
              0.40253142, -1.6318681 , -0.58861333, -0.7433417 ,
              0.5095647 , -0.9508537 ,  0.10090663,  2.105676  ,
             -1.6571679 , -0.38133493, -0.93313324,  0.17004208,
              0.8761585 , -0.31096017, -0.1065815 , -0.08023604],            dtype=float32)}
{'w': DeviceArray([ 1.2244141 , -2.0427547 ,  0.41138938, -0.00851617,
             -0.17035753,  0.7279684 , -0.17386378,  0.8345659 ,
             -0.42434725,  0.12314823, -0.0817145 ,  0.16684777,
              1.082163  , -0.3168161 ,  0.7444031 , -0.28579453,
             -0.7162781 ,  0.9167828 ,  0.3207458 ,  2.6583536 ],            dtype=float32)}
{'w': DeviceArray([ 0.12843032, -0.17618297,  0.01384285, -0.00484724,
             -0.01471046,  0.04710127, -0.00282409,  0.06217055,
             -0.04243679,  0.00889982, -0.02620216,  0.00168099,
              0.13044342, -0.00655541,  0.04094797, -0.01528377,
             -0.

In [93]:
jax.vmap??